In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pygame
from math import acos, degrees

class SnakeGameClass:
    def __init__(self,pathFood):
        self.initializeData()
        self.gameOver = False
        self.score = 0
        
        # Food interaction
        self.imgFood = cv2.imread(pathFood, cv2.IMREAD_UNCHANGED)
        self.hFood, self.wFood, _ = self.imgFood.shape # Image dimension
        
    def randomLocation(self):
        self.foodPoint = random.randint(100, 400), random.randint(100, 400)

    def initializeData(self):
        self.points = [] # Snake points
        self.lenghts = [] # Distance between points
        self.currentLength = 0 # Snake length
        self.allowedLength = 150
        self.previousHead = 0,0
        self.foodPoint = 0,0
        self.randomLocation()
        
    def update(self, imgMain, currentHead):
        font = cv2.FONT_HERSHEY_COMPLEX

        if self.gameOver:
            cv2.putText(imgMain, "Game Over", (50, 80), font, 2, (0, 204, 102), 4)
            cv2.putText(imgMain,f'Your Score {self.score}', (50, 140), font, 2, (0, 204, 102), 4)
        else:
            cx, cy = currentHead
            px, py = self.previousHead
            
            self.points.append((cx, cy))
            distance =  math.hypot(cx - px, cy - py)
            
            self.lenghts.append(distance)
            self.currentLength += distance
            self.previousHead = cx, cy
            self.reduceLength() # Length reduction

            # Check if the snake ate the food
            rx, ry = self.foodPoint
            if(rx-self.wFood//2 < cx < rx + self.wFood//2 and ry - self.hFood//2 < cy < ry + self.hFood//2):
                self.score += 1                
                self.allowedLength += 50
                self.randomLocation()
            
            # Draw the snake
            if(self.points):
                for i, point in enumerate(self.points):
                    if i != 0:
                        cv2.line(imgMain, self.points[i - 1], self.points[i], (0, 0, 255), 15)
                cv2.circle(imgMain, self.points[-1], 15, (0, 0, 153), cv2.FILLED)
            
            # Draw food
            cv2.putText(imgMain, f'Score {self.score}', (50, 80), font, 2, (0, 204, 102), 5)
            imgMain = cvzone.overlayPNG(imgMain, self.imgFood, (rx - self.wFood//2, ry - self.hFood//2))
            
            # Check for collision
            pts = np.array(self.points[:-2], np.int32)
            pts = pts.reshape((-1, 1, 2))
            
            cv2.polylines(imgMain, [pts], False, (0, 200, 0), 2)
            
            distace = cv2.pointPolygonTest(pts, (cx, cy), True)
            if(-1 <= distace <= 1):
                self.gameOver = True
                self.initializeData()
               
        return imgMain
                
    def reduceLength(self)  :
        if(self.currentLength > self.allowedLength):
            for i, lenght in enumerate(self.lenghts):
                # Keep reducing snake's length until it reaches the allowed length
                self.currentLength -= lenght
                self.lenghts.pop(i)
                self.points.pop(i)

                if(self.currentLength < self.allowedLength):
                    break

mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh
mp_hands = mp.solutions.hands

def palm_centroid(coordinates_list):
     coordinates = np.array(coordinates_list)
     centroid = np.mean(coordinates, axis=0)
     centroid = int(centroid[0]), int(centroid[1])
     return centroid

def fingers_up_down(hand_results, thumb_points, palm_points, fingertips_points, finger_base_points, height, width, frame):
     fingers = None
     coordinates_thumb = np.array([]).reshape(0, 2)
     coordinates_palm = np.array([]).reshape(0, 2)
     coordinates_ft = np.array([]).reshape(0, 2)
     coordinates_fb = np.array
     for hand_landmarks in hand_results.multi_hand_landmarks:
          for index in thumb_points:
               x = int(hand_landmarks.landmark[index].x * width)
               y = int(hand_landmarks.landmark[index].y * height)
               coordinates_thumb.append([x, y])
          
          for index in palm_points:
               x = int(hand_landmarks.landmark[index].x * width)
               y = int(hand_landmarks.landmark[index].y * height)
               coordinates_palm.append([x, y])
          
          for index in fingertips_points:
               x = int(hand_landmarks.landmark[index].x * width)
               y = int(hand_landmarks.landmark[index].y * height)
               coordinates_ft.append([x, y])
          
          for index in finger_base_points:
               x = int(hand_landmarks.landmark[index].x * width)
               y = int(hand_landmarks.landmark[index].y * height)
               coordinates_fb.append([x, y])
          ##########################
          # Pulgar
          p1 = np.array(coordinates_thumb[0])
          p2 = np.array(coordinates_thumb[1])
          p3 = np.array(coordinates_thumb[2])
          l1 = np.linalg.norm(p2 - p3)
          l2 = np.linalg.norm(p1 - p3)
          l3 = np.linalg.norm(p1 - p2)
          # Calcular el ángulo
          to_angle = (l1**2 + l3**2 - l2**2) / (2 * l1 * l3)
          if int(to_angle) == -1:
               angle = 180
          else:
               angle = degrees(acos(to_angle))
          thumb_finger = np.array(False)
          if angle > 150:
               thumb_finger = np.array(True)
          
          ################################
          # Índice, medio, anular y meñique
          nx, ny = palm_centroid(coordinates_palm)
          cv2.circle(frame, (nx, ny), 3, (0, 255, 0), 2)
          coordinates_centroid = np.array([nx, ny])
          coordinates_ft = np.array(coordinates_ft)
          coordinates_fb = np.array(coordinates_fb)
          # Distancias
          d_centrid_ft = np.linalg.norm(coordinates_centroid - coordinates_ft, axis=1)
          d_centrid_fb = np.linalg.norm(coordinates_centroid - coordinates_fb, axis=1)
          dif = d_centrid_ft - d_centrid_fb
          fingers = dif > 0
          fingers = np.append(thumb_finger, fingers)
          mp_drawing.draw_landmarks(
               frame,
               hand_landmarks,
               mp_hands.HAND_CONNECTIONS,
               mp_drawing_styles.get_default_hand_landmarks_style(),
               mp_drawing_styles.get_default_hand_connections_style())
     return fingers

def play_snake_game(food_img):
    #food_img = "./minigames/resources/apple.png"
    cap = cv2.VideoCapture(0)
    cap.set(3,1280)
    cap.set(4,1024)

    detector = HandDetector(detectionCon = 0.8, maxHands = 1)
    game = SnakeGameClass(food_img)

    change = False

    hand_detection_active = True 
    # Images to show
    image1 = cv2.imread("images/imagen_inicio.jpg")
    image2 = cv2.imread("images/instruccion_snake.jpg")
    image3 = cv2.imread("images/perder_snake.jpg")
    imAux = image1

    show_instructions = True
    thumb_points = [1, 2, 4]

    # Índice, medio, anular y meñique
    palm_points = [0, 1, 2, 5, 9, 13, 17]
    fingertips_points = [8, 12, 16, 20]
    finger_base_points =[6, 10, 14, 18]

    TO_ACTIVATE = np.array([True, False, False, False, False])
    EXIT = np.array([False, False, False, False, True])

    with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.5) as hand_detector:

        while True:
            success, img = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                continue
            img = cv2.flip(img, 1)
            if hand_detection_active:
                change = False
                height, width, _ = img.shape
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
                results_hands = hand_detector.process(img_rgb)
                if results_hands.multi_hand_landmarks is not None:
                    fingers = fingers_up_down(results_hands, thumb_points, palm_points, fingertips_points, finger_base_points, height, width, img)
                    if not False in (fingers == TO_ACTIVATE):
                        hand_detection_active = False  # Activar la detección de mano al tener el pulgar arriba
                        show_instructions = False
                    elif not False in (fingers == EXIT):
                            break
            else:
                if change:
                    hands, img = detector.findHands(img, flipType  = False, draw=False)
                    if hands: # If a hand is detected
                        lmList = hands[0]["lmList"]
                        pointIndex = lmList[8][0:2]
                        img = game.update(img, pointIndex)  # Update score & snake's length
                        if game.gameOver:
                            show_instructions = True
                            imAux = image3
                            hand_detection_active = True
                            game.gameOver = False
                            game.score = 0
                else:
                    time.sleep(0.5)
                    change = True
            
            resized_image = cv2.resize(img, (800, 600))
            if not show_instructions:
                imAux = image2
            if imAux.shape[0] != resized_image.shape[0]:
                imAux = cv2.resize(imAux, (resized_image.shape[1], resized_image.shape[0]))
            n_image = cv2.hconcat([imAux, resized_image])
            cv2.imshow("Snake Game", n_image)

            key = cv2.waitKey(5)    
            if key == ord('r'):
                game.gameOver = False
                game.score = 0
            if key == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
